# Grade Processing
This notebook is used for generating .csv files which contains learners' weekly grades.

## Getting learners' grading information from DelftX database
We use following SQL statements getting grading information of all passers. The generated table is named as "EX101x_3T2015_pass_student_allrecords.csv"

```sql
SELECT 
	assessments.course_learner_id as course_learner_id, 
	assessments.grade as grade, 
	assessments.max_grade as max_grade,
	questions.question_weight as question_weight,
	submissions.question_id as question_id,
	course_elements.week as relevant_week,
	submissions.submission_timestamp as submission_timestamp
FROM 
	DelftX.assessments AS assessments
	JOIN DelftX.submissions AS submissions
	ON assessments.assessment_id = submissions.submission_id
	JOIN DelftX.quiz_questions AS questions
	ON submissions.question_id = questions.question_id
    JOIN course_elements
    ON course_elements.element_id = questions.question_id
WHERE
	assessments.course_learner_id IN (
		SELECT 
			course_learner_id 
		FROM 
			DelftX.course_learner
		WHERE 
			DelftX.course_learner.certificate_status <> "notpassing")
	AND questions.question_weight > 0
	AND assessments.grade > 0
	AND assessments.max_grade > 0
    AND course_elements.course_id = "course-v1:DelftX+EX101x+3T2015"
```

In [104]:
import pandas as pd
import numpy as np

## Read .csv files generated by previous SQL query

In [141]:
# EX101x relevant csv
grade_allrecords = "EX101x_3T2015_pass_student_allrecords.csv"
course_element_replace = ""
max_score_byweek = [14.0, 17.0, 23.0, 31.0, 25.5, 18.5, 30.0, 31.0]

# # FP101x relevant csv
# grade_allrecords = "FP101x_3T2015_pass_student_allrecords.csv"
# course_element_replace = ""
# max_score_byweek = [9, 45, 53, 45, 43, 18, 50, 23]

# # CTB3365STx
# grade_allrecords = "CTB3365STx_1T2016_pass_student_allrecords.csv"
# course_element_replace = "CTB3365STx_1T2016_GradeAssignList.csv"
# max_score_byweek = [8, 15, 7, 12, 6, 13]

# # RI101x
# grade_allrecords = "RI101x_1T2016_pass_student_allrecords.csv"
# course_element_replace = "RI101x_1T2016_chapter_split.csv"
# max_score_byweek = [6, 11, 10, 12, 16, 12, 14]

## Merge and aggregate grades by week for each learner

In [142]:
df_grade_allrecords = pd.read_csv(grade_allrecords)
df_grade_allrecords.rename(columns={'question_id': 'element_id'}, inplace=True)
# print df_grade_allrecords.head(10)

In [143]:
# remove redundant records, since edx only count the last submission of each learner on each problem
df_grade_allrecords = df_grade_allrecords.sort_values(['submission_timestamp'], ascending=[True])
df_grade_allrecords.reset_index(drop=True, inplace=True)
df_grade_allrecords = df_grade_allrecords.drop_duplicates(['course_learner_id', 'element_id'], keep='last')
# print df_grade_allrecords.iloc[2]['element_id']
# print df_grade_allrecords.iloc[3]['element_id']
print df_grade_allrecords.head(10)

                         course_learner_id  grade  max_grade  question_weight  \
1   course-v1:DelftX+EX101x+3T2015_1243559      2          2              2.0   
2   course-v1:DelftX+EX101x+3T2015_1243559      1          1              1.0   
3   course-v1:DelftX+EX101x+3T2015_1243559      1          1              1.0   
4   course-v1:DelftX+EX101x+3T2015_1243559      8          8              3.0   
5   course-v1:DelftX+EX101x+3T2015_7920693      2          2              2.0   
6   course-v1:DelftX+EX101x+3T2015_1243559      1          1              1.0   
11  course-v1:DelftX+EX101x+3T2015_7920693      8          8              3.0   
12  course-v1:DelftX+EX101x+3T2015_7920693      1          1              1.0   
13  course-v1:DelftX+EX101x+3T2015_7920693      1          1              1.0   
14  course-v1:DelftX+EX101x+3T2015_4273302      2          2              2.0   

                                           element_id  relevant_week  \
1   block-v1:DelftX+EX101x+3T2015+ty

In [144]:
# tune relevant week of course count
if course_element_replace != "":
    df_grade_allrecords.drop('relevant_week', axis=1, inplace=True)
    df_course_element = pd.read_csv(course_element_replace)
    print df_course_element.head(10)
    df_grade_allrecords = df_grade_allrecords.merge(df_course_element, on='element_id')
print df_grade_allrecords.head(10)

                         course_learner_id  grade  max_grade  question_weight  \
1   course-v1:DelftX+EX101x+3T2015_1243559      2          2              2.0   
2   course-v1:DelftX+EX101x+3T2015_1243559      1          1              1.0   
3   course-v1:DelftX+EX101x+3T2015_1243559      1          1              1.0   
4   course-v1:DelftX+EX101x+3T2015_1243559      8          8              3.0   
5   course-v1:DelftX+EX101x+3T2015_7920693      2          2              2.0   
6   course-v1:DelftX+EX101x+3T2015_1243559      1          1              1.0   
11  course-v1:DelftX+EX101x+3T2015_7920693      8          8              3.0   
12  course-v1:DelftX+EX101x+3T2015_7920693      1          1              1.0   
13  course-v1:DelftX+EX101x+3T2015_7920693      1          1              1.0   
14  course-v1:DelftX+EX101x+3T2015_4273302      2          2              2.0   

                                           element_id  relevant_week  \
1   block-v1:DelftX+EX101x+3T2015+ty

In [145]:
# Calculate real grade by question_weight * grade/max_grade
df_grade_allrecords[['question_weight', 'grade', 'max_grade']] = df_grade_allrecords[['question_weight', 'grade', 'max_grade']].astype(float)
df_grade_allrecords['real_grade'] = df_grade_allrecords['question_weight'] * df_grade_allrecords['grade'] / df_grade_allrecords['max_grade']

In [146]:
# Aggregation
df_grade_allrecords = df_grade_allrecords[['course_learner_id','relevant_week','real_grade']]
print df_grade_allrecords.head(10)

                         course_learner_id  relevant_week  real_grade
1   course-v1:DelftX+EX101x+3T2015_1243559              1         2.0
2   course-v1:DelftX+EX101x+3T2015_1243559              1         1.0
3   course-v1:DelftX+EX101x+3T2015_1243559              1         1.0
4   course-v1:DelftX+EX101x+3T2015_1243559              1         3.0
5   course-v1:DelftX+EX101x+3T2015_7920693              1         2.0
6   course-v1:DelftX+EX101x+3T2015_1243559              1         1.0
11  course-v1:DelftX+EX101x+3T2015_7920693              1         3.0
12  course-v1:DelftX+EX101x+3T2015_7920693              1         1.0
13  course-v1:DelftX+EX101x+3T2015_7920693              1         1.0
14  course-v1:DelftX+EX101x+3T2015_4273302              1         2.0


In [147]:
# df_grade_allrecords = df_grade_allrecords.groupby(by = ['course_learner_id', 'relevant_week'])['real_grade'].sum()
df_grade_allrecords = df_grade_allrecords.groupby(['course_learner_id', 'relevant_week'], as_index=False).sum()
# print df_grade_allrecords.head(10)

## Change the shape of the dataframe

In [148]:
# replace relevant week values
df_grade_allrecords['relevant_week'] = df_grade_allrecords['relevant_week'].astype(str)
replace_week_name = lambda x: 'Week_' + x
df_grade_allrecords['relevant_week'] = df_grade_allrecords['relevant_week'].apply(replace_week_name)
# print df_grade_allrecords.head(10)

In [149]:
learner_grade_byweek = df_grade_allrecords.pivot(index='course_learner_id', columns='relevant_week', values='real_grade')
learner_grade_byweek = learner_grade_byweek.fillna(0)
# print learner_grade_byweek.head(10)

In [150]:
# output
course_id = grade_allrecords.split("_")[0] + "_" + grade_allrecords.split("_")[1]
print learner_grade_byweek.head(10)
outputfile = course_id + "_grade_byweek.csv"
learner_grade_byweek.to_csv(outputfile)

relevant_week                           Week_1  Week_2  Week_3     Week_4  \
course_learner_id                                                           
course-v1:DelftX+EX101x+3T2015_1021198  12.000    15.0    23.0  31.000000   
course-v1:DelftX+EX101x+3T2015_1023643  14.000    17.0    23.0  29.400000   
course-v1:DelftX+EX101x+3T2015_1024625  14.000    17.0    21.0  31.000000   
course-v1:DelftX+EX101x+3T2015_1029720  13.000    15.0    23.0  30.000000   
course-v1:DelftX+EX101x+3T2015_1030769  14.000    17.0    23.0  31.000000   
course-v1:DelftX+EX101x+3T2015_1034392  14.000    17.0    23.0  30.333333   
course-v1:DelftX+EX101x+3T2015_10394    13.000    17.0    20.0  30.000000   
course-v1:DelftX+EX101x+3T2015_1053433  13.625    14.0    20.0  23.400000   
course-v1:DelftX+EX101x+3T2015_1055102   7.000    17.0    23.0  31.000000   
course-v1:DelftX+EX101x+3T2015_1060759  13.500    16.0    18.0  31.000000   

relevant_week                           Week_5  Week_6  Week_7     Week_8  

In [151]:
# print learner_grade_byweek.columns.values.tolist()
def learner_grade_alt(df, max_score_byweek):
    learner_grade_byweek_alt = df
    weeklist = learner_grade_byweek_alt.columns.values.tolist()
    for (week, maxgrade) in zip(weeklist, max_score_byweek):
        learner_grade_byweek_alt[week] = learner_grade_byweek_alt[week] / maxgrade
    return learner_grade_byweek_alt

print learner_grade_byweek.columns.values.tolist()
learner_grade_byweek_alt = learner_grade_byweek
learner_grade_byweek_alt = learner_grade_alt(learner_grade_byweek, max_score_byweek)
# print learner_grade_byweek_alt.head(10)

['Week_1', 'Week_2', 'Week_3', 'Week_4', 'Week_5', 'Week_6', 'Week_7', 'Week_8']


In [152]:
print learner_grade_byweek_alt.head(10)
outputfile_alt = course_id + "_grade_byweek_alt.csv"
learner_grade_byweek_alt.to_csv(outputfile_alt)

relevant_week                             Week_1    Week_2    Week_3  \
course_learner_id                                                      
course-v1:DelftX+EX101x+3T2015_1021198  0.857143  0.882353  1.000000   
course-v1:DelftX+EX101x+3T2015_1023643  1.000000  1.000000  1.000000   
course-v1:DelftX+EX101x+3T2015_1024625  1.000000  1.000000  0.913043   
course-v1:DelftX+EX101x+3T2015_1029720  0.928571  0.882353  1.000000   
course-v1:DelftX+EX101x+3T2015_1030769  1.000000  1.000000  1.000000   
course-v1:DelftX+EX101x+3T2015_1034392  1.000000  1.000000  1.000000   
course-v1:DelftX+EX101x+3T2015_10394    0.928571  1.000000  0.869565   
course-v1:DelftX+EX101x+3T2015_1053433  0.973214  0.823529  0.869565   
course-v1:DelftX+EX101x+3T2015_1055102  0.500000  1.000000  1.000000   
course-v1:DelftX+EX101x+3T2015_1060759  0.964286  0.941176  0.782609   

relevant_week                             Week_4    Week_5    Week_6  \
course_learner_id                                              

## Next Step
Next step should be grouping learners into different groups based on week they passed the course. Since different courses do have really different grading schedules and passing conditions. We need to manully coding the grouping method one by one.